In [ ]:
import pandas as pd
import numpy as np
import json
import csv
#1.确定这几个数据都是干什么的
swiping_card_dataframe = pd.read_csv('./bus_swiping_card_info1.csv')
card_info_dataframe = pd.read_csv('./card_info.csv')
bus_swip_dataframe = pd.read_excel('公交数据.xlsx',sheet_name='刷卡数据',header=0,engine='openpyxl')
bus_station_dataframe = pd.read_excel('公交数据.xlsx',sheet_name='站点数据',header=0,engine='openpyxl')
#swiping_card_dataframe ： 每次刷卡的信息：几号线，第几站，车辆的id，刷卡的id，刷卡的时间
#card_info_dataframe ： 每次刷卡的信息：几号线，第几站，车辆的id，刷卡的id，刷卡的时间也是上面的信息，但是时间是去年的。
#用去年的时间的同比数据，去预测今年的同比数据，看是否一致，如果一致，说明经济发展没有变化，如果变大，说明经济好转
#bus_swip_dataframe ：一天的统计信息：各条线路、各个站点的刷卡数量信息
#bus_station_dataframe ： 站点的统计信息：各个站点的经纬度
line_station_number_sequen_dataframe = \
bus_station_dataframe.groupby(['line_identity','station_number']).agg({'station_name':list,'station_identity':list}).reset_index()
line_station_number_sequen_dataframe['length'] = line_station_number_sequen_dataframe['station_identity'].apply(lambda x:len(x))
line_station_number_sequen_dataframe

In [ ]:
line_station_number_sequen_dataframe['id_station'] = \
line_station_number_sequen_dataframe.apply(lambda x:list(zip(x['station_identity'],x['station_name'])),axis=1)
line_station_number_sequen_dataframe
#构建一个字典：
#road_structure = {line_id:{'stations_nums':int,'stations':{station_id:station_name},'all_stations':[]}}
road_line_station_structure_setting={}
for row_index in range(len(line_station_number_sequen_dataframe)):
    line_id = line_station_number_sequen_dataframe[row_index:row_index+1]['line_identity'].values[0]
    stations_nums = line_station_number_sequen_dataframe[row_index:row_index+1]['station_number'].values[0]
    all_stations = line_station_number_sequen_dataframe[row_index:row_index+1]['station_name'].values[0]
    stations={}
    for instance in line_station_number_sequen_dataframe[row_index:row_index+1]['id_station'].values[0]:
        instance_id = instance[0]
        instance_name = instance[1]
        stations[instance_id]=instance_name
    road_line = {'stations_nums':stations_nums,'stations':stations,'all_stations':all_stations}
    road_line_station_structure_setting[line_id] = road_line    
road_line_station_structure_setting  

In [ ]:
station_distance_rank_index_list = np.loadtxt('./station_pair_distance_rank_index.csv',delimiter=',',dtype=int)
# 从文件中读取字典
with open('./id_2_station.json', 'r') as file:
    id_2_station = json.load(file)
# 将字典保存到文件中
with open('./station_2_id.json', 'r') as file:
    station_2_id = json.load(file)
#从card_info_dataframe中找出所有用户一天内的od对信息
card_info_dataframe['custom_day'] = card_info_dataframe['custom_time'].apply(lambda x:x.split(' ')[0])
card_info_dataframe['custom_precise_time'] = card_info_dataframe['custom_time'].apply(lambda x:x.split(' ')[1])
#sum(card_info_dataframe['partitionday']==card_info_dataframe['custom_day'])
#card_info_dataframe
card_info_dataframe.drop(labels=['custom_time','card_type','consume','data_src','data_load_time','partitionday'],axis=1,inplace=True)
card_info_dataframe

In [ ]:
day_0929_dataframe = card_info_dataframe[card_info_dataframe['custom_day']=='2023-09-29']
day_0929_dataframe

# 各个站点为起点站的数量

In [ ]:
#以各个站为起点的刷卡数量统计
day_0929_dataframe_groupby_card_id=day_0929_dataframe.groupby('card_identity'). \
agg({'station_id':list,'custom_precise_time':list,'line_identity':list}).reset_index()
day_0929_dataframe_groupby_card_id
day_0929_dataframe_groupby_card_id['station_time'] = \
day_0929_dataframe_groupby_card_id.apply(lambda x: \
                                        list(zip(x['station_id'],x['custom_precise_time'])),axis=1)
day_0929_dataframe_groupby_card_id
def early(x):
    sorted_x = sorted(x,key=lambda t:t[1],reverse=False)
    return sorted_x[0]
day_0929_dataframe_groupby_card_id['station_time']= \
day_0929_dataframe_groupby_card_id['station_time'].apply(lambda x:early(x))
day_0929_dataframe_groupby_card_id
day_0929_dataframe_groupby_card_id['first_station']=day_0929_dataframe_groupby_card_id['station_time'].apply(lambda x:x[0])
day_0929_dataframe_groupby_card_id
day_0929_dataframe_groupby_card_id_first_station = \
day_0929_dataframe_groupby_card_id.groupby('first_station').agg({'card_identity':list}).reset_index()
day_0929_dataframe_groupby_card_id_first_station
day_0929_dataframe_groupby_card_id_first_station['nums']= \
day_0929_dataframe_groupby_card_id_first_station.apply(lambda x:len(x['card_identity']),axis=1)
day_0929_dataframe_groupby_card_id_first_station

In [ ]:
day_0929_dataframe_groupby_card_id_first_station_swip_times = list(zip(day_0929_dataframe_groupby_card_id_first_station['first_station'], \
         day_0929_dataframe_groupby_card_id_first_station['nums']))
day_0929_dataframe_groupby_card_id_first_station_swip_times
sorted_day_0929_dataframe_groupby_card_id_first_station_swip_times= \
sorted(day_0929_dataframe_groupby_card_id_first_station_swip_times,key=lambda x:x[1],reverse=True)
sorted_day_0929_dataframe_groupby_card_id_first_station_swip_times

In [ ]:
first_time_stations=[]
for i in range(1,5):
    temp={'station':id_2_station[str(sorted_day_0929_dataframe_groupby_card_id_first_station_swip_times[i][0])], \
          'nums':sorted_day_0929_dataframe_groupby_card_id_first_station_swip_times[i][1]}
    first_time_stations.append(temp)
print(first_time_stations)